In [85]:
import pandas as pd
import numpy as np

## Innlesing og opprydding i data

In [239]:
data = pd.read_csv('../../datasets/imdb-5000-movie-dataset/movie_metadata.csv')

to_use = ['genres', 'plot_keywords', 'movie_title', 'actor_1_name', 'actor_2_name', 'actor_3_name', 'director_name', 'imdb_score']
data_use = data[to_use].copy()
data_use['movie_title'] = [i.replace("\xa0","") for i in list(data_use['movie_title'])]

In [240]:
clean_data = data_use.dropna(axis=0)
clean_data = clean_data.drop_duplicates(['movie_title'])
clean_data = clean_data.reset_index(drop=True)
clean_data.shape

(4659, 8)

In [241]:
people_list = []
merge_columns = ['actor_1_name', 'actor_2_name', 'actor_3_name', 'director_name']
for i in range(clean_data.shape[0]):
    people_list.append('|'.join(clean_data.iloc[i][col].replace(' ', '_') for col in merge_columns))
clean_data['people'] = people_list

clean_data.head()

,genres,plot_keywords,movie_title,actor_1_name,actor_2_name,actor_3_name,director_name,imdb_score,people
0,Action|Adventure|Fantasy|Sci-Fi,avatar|future|marine|native|paraplegic,Avatar,CCH Pounder,Joel David Moore,Wes Studi,James Cameron,7.9,CCH_Pounder|Joel_David_Moore|Wes_Studi|James_C...
1,Action|Adventure|Fantasy,goddess|marriage ceremony|marriage proposal|pi...,Pirates of the Caribbean: At World's End,Johnny Depp,Orlando Bloom,Jack Davenport,Gore Verbinski,7.1,Johnny_Depp|Orlando_Bloom|Jack_Davenport|Gore_...
2,Action|Adventure|Thriller,bomb|espionage|sequel|spy|terrorist,Spectre,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Sam Mendes,6.8,Christoph_Waltz|Rory_Kinnear|Stephanie_Sigman|...
3,Action|Thriller,deception|imprisonment|lawlessness|police offi...,The Dark Knight Rises,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Christopher Nolan,8.5,Tom_Hardy|Christian_Bale|Joseph_Gordon-Levitt|...
4,Action|Adventure|Sci-Fi,alien|american civil war|male nipple|mars|prin...,John Carter,Daryl Sabara,Samantha Morton,Polly Walker,Andrew Stanton,6.6,Daryl_Sabara|Samantha_Morton|Polly_Walker|Andr...


Som variabler her, skal vi bruke om hver av sjangerne, keywords eller person har noe med filmen å gjøre. Altså 1 hvis den er der, og 0 ellers.
Avstanden mellom to filmer blir dermed liten hvis de har mange av disse felles.

In [245]:
from sklearn.feature_extraction.text import CountVectorizer

def token(text):
    return(text.split('|'))

cv_kw=CountVectorizer(max_features=100, tokenizer=token)
keywords = cv_kw.fit_transform(clean_data['plot_keywords'])
keywords_list = ['kw_' + i for i in cv_kw.get_feature_names()]

cv_ge=CountVectorizer(tokenizer=token )
genres = cv_ge.fit_transform(clean_data['genres'])
genres_list = ['genres_' + i for i in cv_ge.get_feature_names()]

cv_pp=CountVectorizer(max_features=100,tokenizer=token )
people = cv_pp.fit_transform(clean_data['people'])
people_list = ['pp_' + i for i in cv_pp.get_feature_names()]

cluster_data = np.hstack([keywords.todense(),genres.todense(),people.todense()*2])
criterion_list = keywords_list+genres_list+people_list

## Clustering med K-Means

In [246]:
from sklearn.cluster import KMeans

mod = KMeans(n_clusters=100)
category = mod.fit_predict(cluster_data)
category_dataframe = pd.DataFrame({'category': category}, index=clean_data['movie_title'])

In [247]:
clean_data.iloc[list(category_dataframe['category'] == 0)][['genres', 'movie_title', 'people']]

,genres,movie_title,people
3,Action|Thriller,The Dark Knight Rises,Tom_Hardy|Christian_Bale|Joseph_Gordon-Levitt|...
43,Action|Adventure|Sci-Fi,Terminator Salvation,Christian_Bale|Bryce_Dallas_Howard|Common|McG
157,Action|Adventure|Drama,Exodus: Gods and Kings,Christian_Bale|María_Valverde|Ben_Mendelsohn|R...
317,Drama|History|Romance|War,The Flowers of War,Christian_Bale|Ni_Ni|Shigeo_Kobayashi|Yimou_Zhang
744,Action|Adventure|Fantasy|Sci-Fi|Thriller,Reign of Fire,Christian_Bale|Gerard_Butler|Matthew_McConaugh...
804,Drama|Music|Romance|War,Captain Corelli's Mandolin,Christian_Bale|Irene_Papas|Mihalis_Giannatos|J...
810,Adventure|Animation|Drama|Family|History|Music...,Pocahontas,Christian_Bale|Frank_Welker|Irene_Bedard|Mike_...
871,Action|Crime|Thriller,Shaft,Christian_Bale|Vanessa_Williams|Daniel_von_Bar...
1017,Adventure|Crime|Drama|Western,3:10 to Yuma,Christian_Bale|Logan_Lerman|Gretchen_Mol|James...
1644,Biography|Drama|History|Romance,The New World,Christian_Bale|Michael_Greyeyes|Wes_Studi|Terr...


## Anbefaling av filmer

Vi kan bruke `recommend`-funksjonen til å få filmer som ligner på den vi sender inn. Filmtitlene kommer ut sortert etter IMDb-score.

In [248]:
def recommend(movie_name, recommend_number=5):
    if movie_name in list(clean_data['movie_title']):
        movie_cluster = category_dataframe.loc[movie_name]['category']
        score = clean_data.iloc[list(category_dataframe['category'] == movie_cluster)][['imdb_score', 'movie_title']]
        sort_score = score.sort_values(['imdb_score'], ascending=[0])
        sort_score = sort_score[sort_score['movie_title'] != movie_name]
        recommend_number = min(sort_score.shape[0], recommend_number)
        recommend_movie = list(sort_score.iloc[range(recommend_number), 1])
        return recommend_movie
    else:
        print('Can\'t find this movie!')

In [249]:
recommend('Harry Potter and the Chamber of Secrets', 10)

['Spirited Away',
 'How to Train Your Dragon',
 'The Wizard of Oz',
 'The Princess Bride',
 'Willy Wonka & the Chocolate Factory',
 'Harry Potter and the Prisoner of Azkaban',
 'Snow White and the Seven Dwarfs',
 'The Secret of Kells',
 'Ponyo',
 'Coraline']

**Oppgave:** Kan du komme på en bedre måte å sortere dataene på?

Løsning: eks. sortere etter avstand til filmen i clusteret